## LLM: Attention Mechanism

4 types of attention mechanism
1. Simplified attention
    - simplified self attention technique
2. Self attention
    - self attention with trainable weights
3. Casual attention
    - self attention that allows model to only consider previous and current inputs in a sequence
4. Multihead attention
    - an extension of self attention and causal attention that enables the model to simultaneously attend the information from different representative subspace
    - LLM attends to various input data in parallel


The problem with modeling long sequence:
- in case of language translation, translation process require contextual understanding and grammar alignment. 
- this is not possible when we try to perform word to word translation.
- One solution was to utilise a neural network with two sub modules, an encoder and a decoder
    - encoder reads and processes the text into a context vector
    - decoder translate the text using the context vector 
- before transformers, RNN were used the most popular encoder and decoder architecture
    - RNN or recurrent neural network: output from previous step is fed as input to the current model

steps of RNN encoder decoder architecture 
1. Input text
2. Enoder (processes input text sequentially)
3. Updates hidden state at each step ( internal values at hidden layers )
4. final hidden state ( encoder tries to capture sentence meaning )
5. Decoder uses this final hidden state to generate translated sentence ( decoder also updates its hidden state at each time )

- the encoder processes the entire tezt into a hidden state ( memory cell ). Decoder takes this hidden state to produce an output
- Big Issue: 
    - RNN cannot directly access earlier hidden states from the encoder during the encoding phase
    - it relies solely on the current hidden state
    - this lead to a loss of context, especially in complex sentences where dependencies might span long distance
    - Difficult for RNN to capture all information in a single vector
- Capturing data dependencies with attention mechanism 
- in RNN, the decoder does not have previous words as input along with the context vector
